# 批处理

每次迭代，我们用一个样本进行训练。这种方式称为**随机梯度下降**（Stochastic Gradient Descent, SGD）。虽然这种方式简单，但是无法充分利用 NumPy 的矢量并行计算能力。

实际上，我们可以一次将多个样本送入网络模型进行训练。这种方式称为**批处理**（Batch Processing），是现代深度学习的标准做法。

按照每次使用的样本数量，梯度下降可分为三类：

* **随机梯度下降**（SGD）:每次把**一个样本**送入迭代；
* **批量梯度下降**（Batch GD）：每次把**全部样本**送入迭代。实践中很少使用批量梯度下降进行训练，因为需要大量的内存，运算速度会非常缓慢；
* **小批量梯度下降**（Mini-batch GD）：每次把**数个样本**送入迭代。小批量梯度下降充分地利用了 NumPy（或者其他计算库）的矢量并行计算能力，是目前常规的模型训练方式。

## 收敛

**收敛**（Convergence）指模型训练的过程中，网络模型通过不断的迭代，其损失值不再显著下降，模型参数趋于稳定。

* **随机梯度下降**：每次迭代只能从一个样本进行学习，因此梯度变化随机性较大，收敛方向容易左右摇摆，所以收敛速度较慢；
* **批量梯度下降**：每次迭代都可以从全部数据进行学习，因此可以相对保持正确的收敛方向，所以收敛速度较快；
* **小批量梯度下降**：折中方案，兼顾效率和稳定性。

## 泛化

**泛化**（Generalization）是评价模型训练效果的一个重要指标，即网络模型对新数据的处理能力。简单地讲，就是一个在训练集上表现良好的网络模型，是否可以在测试集上同样有好的表现。

通常来讲，泛化能力来自于对多样化的数据进行学习。因此：

* **随机梯度下降**：每次一个样本的训练方式，通常可以带来更好的模型泛化能力；
* **批量梯度下降**：每次训练的数据一样，缺少变化，会影响泛化能力的提高；
* **小批量梯度下降**：折中方案，兼顾多样性和稳定性。

## 拟合

**拟合**（fitting）是指网络模型在训练集上的表现，即网络模型学习数据规律的程度。

泛化能力差一般就是因为拟合不理想，通常有两种情况：

* **欠拟合**（Underfitting）：网络模型在训练集上表现就很差，在测试集上表现也很差。通常是因为训练不足，或者网络模型过于简单。
* **过拟合**（Overfitting）：网络模型在训练集上接近优异，但在测试集上表现糟糕。通常是因为网络模型过于复杂，学习到了多余的、没有代表性的细节。

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 小批量梯度下降同时具有较好的收敛速度和泛化能力，因此在实际工作中是最常用的训练方式。
</div>

In [97]:
import numpy as np

## 数据集

### 训练数据：特征、标签

In [98]:
train_features = np.array([[22.5, 72.0],
                           [31.4, 45.0],
                           [19.8, 85.0],
                           [27.6, 63.0]])
train_labels = np.array([[95],
                        [210],
                        [70],
                        [155]])

### 测试数据：特征、标签

In [99]:
test_features = np.array([[28.1, 58.0]])
test_labels = np.array([[165]])

## 模型

### 参数：权重、偏置

In [100]:
weight = np.ones((1, 2)) / 2
bias = np.zeros(1)

### 推理函数

In [101]:
def forward(x, w, b):
    a = x @ w.T + b
    return a

### 损失函数（均方误差）

In [102]:
def mse_loss(p, y):
    return np.mean(np.square(y - p))

### 梯度函数

批处理中，我们在梯度计算时需要处理多个样本。

完整的均方误差损失函数的公式是：

$$
L = \frac{1}{n} \sum\limits_{i=1}^{n} (y_i - p_i)^2
$$

完整的误差项的公式是：

$$
\delta = -\frac{2}{n} (y - p)
$$

这里， $n$ 是每个批处理里包括的样本数量。

因此，批处理的梯度函数需要在计算时，除以样本数量，以避免造成梯度叠加。

In [103]:
def gradient(p, y):
    return - 2 * (y - p) / len(y)

### 反向函数

批处理的梯度函数将计算出多个误差项，每个误差项对应一个样本。

批处理的反向函数中，可以利用 NumPy 的矢量运算，将各个样本对权重的梯度贡献在**批的维度**（第一个维度）上进行加权求和，从而得到权重参数的梯度。

对于偏置，其梯度可以通过在**批的维度**上对误差项进行求和得到。

In [104]:
def backward(x, d, w, b, lr):
    w = w - d.T @ x * lr
    b = b - np.sum(d, axis=0) * lr
    return w, b

## 训练

### 超参数：学习率

In [105]:
LEARNING_RATE = 0.00001

### 超参数：批大小

作为我们的第二个超参数，**批大小**（Batch Size）定义了每个批处理所使用的样本数量。

我们将采用小批量梯度下降算法。但是通过调整批大小，同样可以实现随机梯度下降和批量梯度下降。

<div style="border-left: 4px solid #4CAF50; background:#f9f9f9; padding:10px; margin:10px 0;">
<strong>💡 提示：</strong> 实践中，批大小通常选择 2 的幂次方（如16, 32, 64, 128...）。这不是数学要求，而是因为这样设置能最大化硬件的运行效率。
</div>

In [106]:
BATCH_SIZE = 2

### 迭代

采用小批量梯度下降后，训练集中的 4 个样本，一轮只需要 2 次迭代。

In [107]:
for i in range(0, len(train_features), BATCH_SIZE):
    # 每次读入一个批次需要的多个训练数据：特征、标签
    features = train_features[i: i + BATCH_SIZE]
    labels = train_labels[i: i + BATCH_SIZE]

    # 同时对多个训练数据进行前向传播
    predictions = forward(features, weight, bias)
    # 计算多个训练数据的梯度
    delta = gradient(predictions, labels)
    # 反向喜欢薄（用梯度平均值更新参数）
    weight, bias = backward(features, delta, weight, bias, LEARNING_RATE)

print(f"weight: {weight}")
print(f"bias: {bias}")

weight: [[0.59388172 0.68104165]]
bias: [0.00327249]


## 验证

### 推理

In [108]:
predictions = forward(test_features, weight, bias)
print(f'predictions: {predictions}')

predictions: [[56.19176426]]


### 评估

In [109]:
error = mse_loss(predictions, test_labels)
print(f'loss: {error}')

loss: 11839.232164432306


从验证的结果看，小批量梯度下降一轮迭代的次数变少，但是收敛效果也相应地不足。

## 总结

| 方式                     | 批大小  |                      收敛速度 | 泛化能力 | 过拟合风险 |
|------------------------|:----:|--------------------------:|-----:|------:|
| 随机梯度下降（SGD）            | 一个样本 |                    慢（震荡大） |    强 |     低 |
| 批量梯度下降（Batch GD）       | 全部样本 | 快（单步稳定）<br/>但硬件（内存与算力）要求高 |    弱 |     高 |
| 小批量梯度下降（Mini-batch GD） | 数个样本 |                        中等 |    中等 |     中等 |

## 课后练习

尝试一下批量梯度下降的效果？